In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import time

#Import and read csvs - please note that the csv files from kaggle were titled using the year the report was published.
#The date on the df title reflects the year the actual data applies to (i.e. the happiness2020.csv actually contains 2019 data)
file2018 = "happiness2018.csv"
file2019 = "happiness2019.csv"
file2020 = "happiness2020.csv"

hap2017_df = pd.read_csv(file2018)
hap2018_df = pd.read_csv(file2019)
hap2019_df = pd.read_csv(file2020)


In [2]:
#renaming "Country or region" in original 2018 file to "Country" for ease of merging
hap2017_df = hap2017_df.rename(columns={"Country or region": "Country"})

#drop unecessary columns:
clean_2017_df = hap2017_df.drop(columns = ["Overall rank"])
clean_2017_df

,Country,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,Finland,7.632,1.305,1.592,0.874,0.681,0.202,0.393
1,Norway,7.594,1.456,1.582,0.861,0.686,0.286,0.340
2,Denmark,7.555,1.351,1.590,0.868,0.683,0.284,0.408
3,Iceland,7.495,1.343,1.644,0.914,0.677,0.353,0.138
4,Switzerland,7.487,1.420,1.549,0.927,0.660,0.256,0.357
...,...,...,...,...,...,...,...,...
151,Yemen,3.355,0.442,1.073,0.343,0.244,0.083,0.064
152,Tanzania,3.303,0.455,0.991,0.381,0.481,0.270,0.097
153,South Sudan,3.254,0.337,0.608,0.177,0.112,0.224,0.106
154,Central African Republic,3.083,0.024,0.000,0.010,0.305,0.218,0.038


In [3]:
#renaming "Country or region" in original 2019 file to "Country" for ease of merging
hap2018_df = hap2018_df.rename(columns={"Country or region": "Country"})

#drop unecessary columns:
clean_2018_df = hap2018_df.drop(columns = ["Overall rank"])


#add year column to hap2019_df
#hap2019_df['Year'] = "2017"
#hap2019_df['Year'] = pd.to_numeric(hap2019_df['Year'])


In [4]:
#drop columns we dont need in 2020 data and rename "Country name" column to "Country" for ease of merging
hap2019_df = hap2019_df.rename(columns={"Country name": "Country"})

#drop unecessary columns:
clean_2019_df = hap2019_df.drop(columns = ["Standard error of ladder score", 
                                           "Logged GDP per capita", 
                                           "upperwhisker", 
                                           "lowerwhisker", 
                                           "Ladder score in Dystopia", 
                                           "Dystopia + residual", 
                                           "Social support", 
                                           "Healthy life expectancy", 
                                           "Freedom to make life choices", 
                                           "Generosity", 
                                           "Perceptions of corruption"])


#rename columns:
clean_2019_df = clean_2019_df.rename(columns=
                             {"Explained by: Log GDP per capita":"GDP per capita 2019",
                              "Explained by: Social support":"Social support 2019",
                              "Explained by: Healthy life expectancy": "Healthy life expectancy 2019",
                              "Explained by: Freedom to make life choices": "Freedom to make life choices 2019",
                              "Explained by: Generosity":"Generosity 2019", 
                              "Explained by: Perceptions of corruption": "Perceptions of corruption 2019",
                              "Ladder score": "Score 2019"
                              })


clean_2019_df


,Country,Regional indicator,Score 2019,GDP per capita 2019,Social support 2019,Healthy life expectancy 2019,Freedom to make life choices 2019,Generosity 2019,Perceptions of corruption 2019
0,Finland,Western Europe,7.8087,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857
1,Denmark,Western Europe,7.6456,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260
2,Switzerland,Western Europe,7.5599,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946
3,Iceland,Western Europe,7.5045,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541
4,Norway,Western Europe,7.4880,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101
...,...,...,...,...,...,...,...,...,...
148,Central African Republic,Sub-Saharan Africa,3.4759,0.041072,0.000000,0.000000,0.292814,0.253513,0.028265
149,Rwanda,Sub-Saharan Africa,3.3123,0.343243,0.522876,0.572383,0.604088,0.235705,0.485542
150,Zimbabwe,Sub-Saharan Africa,3.2992,0.425564,1.047835,0.375038,0.377405,0.151349,0.080929
151,South Sudan,Sub-Saharan Africa,2.8166,0.289083,0.553279,0.208809,0.065609,0.209935,0.111157


In [5]:
#ran counts for all dataframes to determine if there was any missing info. none were found:
#clean_2017_df.count()
#clean_2018_df.count()
#clean_2019_df.count()


In [6]:
#analyzed data types for each column in each dataframe. All items are floats except for regional indicator and country name, which makes sense.
#clean_2017_df.dtypes
#clean_2018_df.dtypes
clean_2019_df.dtypes

Country                               object
Regional indicator                    object
Score 2019                           float64
GDP per capita 2019                  float64
Social support 2019                  float64
Healthy life expectancy 2019         float64
Freedom to make life choices 2019    float64
Generosity 2019                      float64
Perceptions of corruption 2019       float64
dtype: object

In [7]:
#no further cleaning of our 2018,2019 and 2020 data is needed. Now we can merge the files. 
merge1_df = pd.merge(clean_2017_df, clean_2018_df, on = "Country", how = "inner", suffixes=(" 2017", " 2018"))
merge1_df

,Country,Score 2017,GDP per capita 2017,Social support 2017,Healthy life expectancy 2017,Freedom to make life choices 2017,Generosity 2017,Perceptions of corruption 2017,Score 2018,GDP per capita 2018,Social support 2018,Healthy life expectancy 2018,Freedom to make life choices 2018,Generosity 2018,Perceptions of corruption 2018
0,Finland,7.632,1.305,1.592,0.874,0.681,0.202,0.393,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,Norway,7.594,1.456,1.582,0.861,0.686,0.286,0.340,7.554,1.488,1.582,1.028,0.603,0.271,0.341
2,Denmark,7.555,1.351,1.590,0.868,0.683,0.284,0.408,7.600,1.383,1.573,0.996,0.592,0.252,0.410
3,Iceland,7.495,1.343,1.644,0.914,0.677,0.353,0.138,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,Switzerland,7.487,1.420,1.549,0.927,0.660,0.256,0.357,7.480,1.452,1.526,1.052,0.572,0.263,0.343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,Yemen,3.355,0.442,1.073,0.343,0.244,0.083,0.064,3.380,0.287,1.163,0.463,0.143,0.108,0.077
148,Tanzania,3.303,0.455,0.991,0.381,0.481,0.270,0.097,3.231,0.476,0.885,0.499,0.417,0.276,0.147
149,South Sudan,3.254,0.337,0.608,0.177,0.112,0.224,0.106,2.853,0.306,0.575,0.295,0.010,0.202,0.091
150,Central African Republic,3.083,0.024,0.000,0.010,0.305,0.218,0.038,3.083,0.026,0.000,0.105,0.225,0.235,0.035


In [8]:
#merge 2019 data into 2017_2018 df:
merge2_df = pd.merge(merge1_df, clean_2019_df, on = "Country", how = "inner")
merge2_df

,Country,Score 2017,GDP per capita 2017,Social support 2017,Healthy life expectancy 2017,Freedom to make life choices 2017,Generosity 2017,Perceptions of corruption 2017,Score 2018,GDP per capita 2018,...,Generosity 2018,Perceptions of corruption 2018,Regional indicator,Score 2019,GDP per capita 2019,Social support 2019,Healthy life expectancy 2019,Freedom to make life choices 2019,Generosity 2019,Perceptions of corruption 2019
0,Finland,7.632,1.305,1.592,0.874,0.681,0.202,0.393,7.769,1.340,...,0.153,0.393,Western Europe,7.8087,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857
1,Norway,7.594,1.456,1.582,0.861,0.686,0.286,0.340,7.554,1.488,...,0.271,0.341,Western Europe,7.4880,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101
2,Denmark,7.555,1.351,1.590,0.868,0.683,0.284,0.408,7.600,1.383,...,0.252,0.410,Western Europe,7.6456,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260
3,Iceland,7.495,1.343,1.644,0.914,0.677,0.353,0.138,7.494,1.380,...,0.354,0.118,Western Europe,7.5045,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541
4,Switzerland,7.487,1.420,1.549,0.927,0.660,0.256,0.357,7.480,1.452,...,0.263,0.343,Western Europe,7.5599,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Yemen,3.355,0.442,1.073,0.343,0.244,0.083,0.064,3.380,0.287,...,0.108,0.077,Middle East and North Africa,3.5274,0.392702,1.177477,0.415000,0.243721,0.094689,0.087352
140,Tanzania,3.303,0.455,0.991,0.381,0.481,0.270,0.097,3.231,0.476,...,0.276,0.147,Sub-Saharan Africa,3.4762,0.457163,0.872675,0.442678,0.509343,0.271541,0.203881
141,South Sudan,3.254,0.337,0.608,0.177,0.112,0.224,0.106,2.853,0.306,...,0.202,0.091,Sub-Saharan Africa,2.8166,0.289083,0.553279,0.208809,0.065609,0.209935,0.111157
142,Central African Republic,3.083,0.024,0.000,0.010,0.305,0.218,0.038,3.083,0.026,...,0.235,0.035,Sub-Saharan Africa,3.4759,0.041072,0.000000,0.000000,0.292814,0.253513,0.028265


In [9]:
merge2_df.count()

#we have decided to keep the 2017 perceptions of corruption cell blank for United Arab Emirates. Further action TBD during analysis.

Country                              144
Score 2017                           144
GDP per capita 2017                  144
Social support 2017                  144
Healthy life expectancy 2017         144
Freedom to make life choices 2017    144
Generosity 2017                      144
Perceptions of corruption 2017       143
Score 2018                           144
GDP per capita 2018                  144
Social support 2018                  144
Healthy life expectancy 2018         144
Freedom to make life choices 2018    144
Generosity 2018                      144
Perceptions of corruption 2018       144
Regional indicator                   144
Score 2019                           144
GDP per capita 2019                  144
Social support 2019                  144
Healthy life expectancy 2019         144
Freedom to make life choices 2019    144
Generosity 2019                      144
Perceptions of corruption 2019       144
dtype: int64

In [10]:
#final dataframe complete
final_df = merge2_df.copy()

#move regional indicator column next to country
cols_to_order = ['Country', 'Regional indicator']
new_columns = cols_to_order + (final_df.columns.drop(cols_to_order).tolist())
final_df = final_df[new_columns]

final_df #matches merge2_df with columns reordered

,Country,Regional indicator,Score 2017,GDP per capita 2017,Social support 2017,Healthy life expectancy 2017,Freedom to make life choices 2017,Generosity 2017,Perceptions of corruption 2017,Score 2018,...,Freedom to make life choices 2018,Generosity 2018,Perceptions of corruption 2018,Score 2019,GDP per capita 2019,Social support 2019,Healthy life expectancy 2019,Freedom to make life choices 2019,Generosity 2019,Perceptions of corruption 2019
0,Finland,Western Europe,7.632,1.305,1.592,0.874,0.681,0.202,0.393,7.769,...,0.596,0.153,0.393,7.8087,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857
1,Norway,Western Europe,7.594,1.456,1.582,0.861,0.686,0.286,0.340,7.554,...,0.603,0.271,0.341,7.4880,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101
2,Denmark,Western Europe,7.555,1.351,1.590,0.868,0.683,0.284,0.408,7.600,...,0.592,0.252,0.410,7.6456,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260
3,Iceland,Western Europe,7.495,1.343,1.644,0.914,0.677,0.353,0.138,7.494,...,0.591,0.354,0.118,7.5045,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541
4,Switzerland,Western Europe,7.487,1.420,1.549,0.927,0.660,0.256,0.357,7.480,...,0.572,0.263,0.343,7.5599,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Yemen,Middle East and North Africa,3.355,0.442,1.073,0.343,0.244,0.083,0.064,3.380,...,0.143,0.108,0.077,3.5274,0.392702,1.177477,0.415000,0.243721,0.094689,0.087352
140,Tanzania,Sub-Saharan Africa,3.303,0.455,0.991,0.381,0.481,0.270,0.097,3.231,...,0.417,0.276,0.147,3.4762,0.457163,0.872675,0.442678,0.509343,0.271541,0.203881
141,South Sudan,Sub-Saharan Africa,3.254,0.337,0.608,0.177,0.112,0.224,0.106,2.853,...,0.010,0.202,0.091,2.8166,0.289083,0.553279,0.208809,0.065609,0.209935,0.111157
142,Central African Republic,Sub-Saharan Africa,3.083,0.024,0.000,0.010,0.305,0.218,0.038,3.083,...,0.225,0.235,0.035,3.4759,0.041072,0.000000,0.000000,0.292814,0.253513,0.028265


In [11]:
#export dataframe to csv
final_df.to_csv(r'Final_DF.csv')